Copyright 2024 Anil Rao

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Getting Started

## Basic Functionality

We firstly do some imports:

In [1]:
from causalite import node_models as nm
from causalite import causal_models as cm
from scipy import stats
import pandas as pd
import numpy as np

### Creating a Structural Causal Model

We will now define a Structural Causal Model (SCM) consisting of 3 variables/nodes A, X and Y with causal mechanisms specified by models f_A, f_X and f_Y respectively. (We use the words 'variable' and 'node' interchangeably.) To do this, we firstly use NodeAdditiveNoiseModel to specify the causal mechanism for each node:


In [2]:
f_A = nm.NodeAdditiveNoiseModel('A')
f_X = nm.NodeAdditiveNoiseModel('X', parent_polys={'A': [1., 0., -3.]})
f_Y = nm.NodeAdditiveNoiseModel('Y', parent_polys={'X': [-0.5, 0., 1.2], 'A': [1.4], 'XA': [3., -1.]})

Here, f_A is the model for the variable A which is a root node (has no parents) in the corresponding Directed Acyclic Graph, and so is distributed according to the exogenous noise U_A:

In [3]:
print(f_A)

A <-  U_A


As f_A models a root node, it is created by specifying the modelled variable, A, and the exogenous noise U_A. By default, this exogenous noise is normally distributed with mean 0 and standard deviation 1, but we can specify alternative distributions if we wish. For example, 



In [4]:
f_A_mean_10_exogenous = nm.NodeAdditiveNoiseModel('A', loc=10.)

specifies that the exogenous noise is normally distributed with mean 10 and standard deviation 1. Alternatively,

In [5]:
f_A_uniform_exogenous = nm.NodeAdditiveNoiseModel('A', u_draw_random_variates=stats.uniform.rvs, loc=2, scale=10)

specifies it is uniformly distributed on \[2, 12\]. We can also specify discrete distributions for the exogenous noise if we choose.

We create models for non root nodes such as X by additionally providing a 'parent_polys' dictionary which specifies the fixed part of their causal relationship with their parent nodes. Each key in 'parent_polys' is either the string name of a parent node or the concatenation of 2 such names. The corresponding value is a list where the ith entry is the coefficient of the i+1th power in a polynomial function of the key. Formally this is:

$q(k) = \sum_{i=0} v[i] * k^{i+1}$

where q is the polynomial, k is the key and v is the value. For node models of type NodeAdditiveNoiseModel, the resulting model is the sum of each of the polynomials and the exogenous noise. Hence the model for causal mechanism f_X is:

In [6]:
print(f_X)

X <-  1.0A - 3.0A^3 + U_X


because its 'parent_polys' dictionary contains 1 item with key 'A' and value \[1.0, 0.0, -3.\]

For f_Y, we include more dictionary items to incorporate polynomial functions of multiple parent nodes into the model:

In [7]:
print(f_Y)

Y <-  - 0.5X + 1.2X^3 + 1.4A + 3.0XA - 1.0(XA)^2 + U_Y


In this case, 'parent_polys' contains keys 'X', 'A' and 'XA'. The key 'XA' represents an interaction between X and A in the model. Currently, only polynomials of single variables and of pairwise interactions between variables can be included.

We can now create an SCM consisting of these causal mechanisms:

In [8]:
model = cm.StructuralCausalModel(node_models=[f_A, f_X, f_Y])
print(model)


Structural Causal Model

A <-  U_A

X <-  1.0A - 3.0A^3 + U_X

Y <-  - 0.5X + 1.2X^3 + 1.4A + 3.0XA - 1.0(XA)^2 + U_Y


### Sampling from the SCM

We can sample from the SCM as follows:

In [9]:
samples = model.draw_sample(size=50000, initial_random_state=0, return_dataframe=True)

Drawing from...
A
X
Y
done


The above will provide 50000 samples from the model in a pandas dataframe.

In [10]:
samples.head()

,A,X,Y
0,1.764052,-13.080164,-3278.515912
1,0.400157,-0.403826,0.115948
2,0.978738,-2.362115,-27.680988
3,2.240893,-32.590699,-47071.333394
4,1.867558,-16.807889,-6768.235113


By specifying the value of 'initial_random_state', we can draw repeatable samples from the SCM.

## Simulating Randomised Controlled Trials

To illustrate how we can use **causalite** to simulate Randomised Controlled Trials (RCTs), we create the following causal model and sample from it:

In [11]:
f_A = nm.NodeAdditiveNoiseModel('A')
f_X = nm.NodeBinaryLogisticModel('X', parent_polys={'A': [1., 0., -3.]})
f_Y = nm.NodeAdditiveNoiseModel('Y', parent_polys={'X': [-0.5, 0., 1.2], 'A': [1.4], 'XA': [3.]})
model = cm.StructuralCausalModel(node_models=[f_A, f_X, f_Y])
samples = model.draw_sample(size=50000, initial_random_state=0, return_dataframe=True)

Drawing from...
A
X
Y
done


Here we used NodeBinaryLogisticModel, rather than NodeAdditiveNoiseModel to create the model f_X. We can use this class to create binary-valued nodes that are Bernoulli distributed by providing the variable name and a 'parent_polys' dictionary. The underlying model adds the polynomial sum specified by 'parent_polys' to logistic exogenous noise and dichotomizes the result. For f_X this is:

In [12]:
print(f_X)

X <-  I( 1.0A - 3.0A^3 + U_X > 0.0 )


where I is the indicator function and U_X ~ logistic(0, 1). This is the 'latent value' formulation of an equivalent Bernoulli model with parameter p, where p equals the sigmoid of the polynomial sum:

$X \leftarrow \text{Bernoulli}(p = \text{sigmoid}(A - 3A^3))$

The resulting SCM is:

In [13]:
print(model)


Structural Causal Model

A <-  U_A

X <-  I( 1.0A - 3.0A^3 + U_X > 0.0 )

Y <-  - 0.5X + 1.2X^3 + 1.4A + 3.0XA + U_Y


We now assign the following meanings to the model variables: A is a subject's age, X indicates whether they have been treated with a specific drug (1=Treated, 0=Untreated), and Y is their outcome. With this interpretation, 'samples' is then data from an *observational* study of drug effect, in which age is a confounding variable. 

We can generate data from an *experimental*, rather than observational, study of drug effect by simulating an RCT as follows:

In [14]:
rct_X_samples = model.draw_rct_sample(size=50000, initial_random_state=0, return_dataframe=True, treatment_variable='X', intervention_draw_random_variates=stats.bernoulli.rvs, p=0.5)

Drawing from...
A
X
Y
done


The above will randomly assign values in {0,1} to the treatment variable X, and draw samples from the resulting model.

In [15]:
rct_X_samples.head()

,A,X,Y
0,1.764052,0.0,2.052915
1,0.400157,1.0,2.404425
2,0.978738,0.0,-0.765963
3,2.240893,0.0,4.777521
4,1.867558,0.0,0.821146


These samples can then be used to estimate the average causal effect of the drug on the outcome in a straightforward manner. We can also use **causalite** to simulate RCTs using other treatment allocation schemes. This is achieved by specifying different 'intervention_draw_random_variates' and distribution parameters when generating the experimental sample.

## Generating do-operator Samples

Do-operator samples are generated by intervening on a variable using a constant intervention distribution. We do this as follows:

In [16]:
do_X_1_samples = model.draw_do_operator_sample(size=50000, initial_random_state=0, return_dataframe=True, intervention_variable='X', intervention_value=1.)

Drawing from...
A
X
Y
done


This will return a sample from 'model' after setting the value of X to be 1 for the whole population.

In [17]:
do_X_1_samples.head()

,A,X,Y
0,1.764052,1.0,8.045072
1,0.400157,1.0,2.404425
2,0.978738,1.0,2.870251
3,2.240893,1.0,12.200201
4,1.867558,1.0,7.123820


## Computing Counterfactuals

We can also use 'model' to compute counterfactuals under an intervention, given a set of observed data. For example

In [18]:
counterfactuals_X_0 = model.compute_counterfactuals(samples, intervention_variable='X', intervention_values=np.zeros(samples.shape[0]), return_dataframe=True)

Abducting exogenous data for...
A
Y
Predicting...
A
Y
done


will compute what the data would have been for each of the individuals in 'samples', if none of them had received drug treatment. Currently, only deterministic counterfactuals are supported.

In [19]:
counterfactuals_X_0.head()

,X,A,Y
0,0.0,1.764052,2.052915
1,0.0,0.400157,0.503953
2,0.0,0.978738,-0.765963
3,0.0,2.240893,4.777521
4,0.0,1.867558,0.821146
